# Implementing A Decision Tree #

In this small notebook, I will try to implement a decision tree from scratch (no XGBoost) on the dataset mentioned in Part 1, to compare accuracy in predictions

In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#Used for impurity measure
def Entropy(y):
    class_counts = np.bincount(y)
    total = len(y)
    probabilities = class_counts / total
    return -np.sum(probabilities * np.log2(probabilities + 1e-9))

def Information_Gain(X,y,feature_index):
    original_entropy = Entropy(y)
    
    feature_values = np.unique(X[:, feature_index])
    
    weighted_entropy = 0
    for value in feature_values:
        mask = X[:, feature_index] == value
        y_subset = y[mask]
        
        # Calculate the weighted entropy 
        weighted_entropy += (len(y_subset) / len(y)) * Entropy(y_subset)
    
    # Information Gain is the reduction in entropy
    return original_entropy - weighted_entropy

def Cut(X,y,feature_index,value):
    mask = X[:, feature_index] == value
    return X[mask], y[mask]

def Best_Cut(X,y):
    best_info_gain = -float('inf')
    best_split_info = None
    n_features = X.shape[1]
    
    # Iterate over all features
    for feature_index in range(n_features):
        # Calculate Information Gain for each feature
        info_gain = Information_Gain(X, y, feature_index)
        
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_split_info = feature_index
    
    return best_split_info

# Recursive function to build the tree (For this part I did use ChatGPT)
def build_tree(X, y, depth=0, max_depth=5):
    # Stopping conditions
    if len(np.unique(y)) == 1:  # All samples are the same class
        return {'label': np.unique(y)[0]}
    if depth >= max_depth:  # Max depth reached
        return {'label': np.bincount(y).argmax()}  # Majority class
    
    # Find the best split (highest Information Gain)
    best_feature_index = Best_Cut(X, y)
    best_feature_values = np.unique(X[:, best_feature_index])
    
    tree = {'feature_index': best_feature_index, 'children': {}}
    
    # Split data and create child nodes
    for value in best_feature_values:
        X_left, y_left = Cut(X, y, best_feature_index, value)
        
        # Recursively build the tree for each subset
        tree['children'][value] = build_tree(X_left, y_left, depth + 1, max_depth)
    
    return tree


def predict(tree, X):
    if 'label' in tree:  
        return tree['label']
    
  
    feature_value = X[tree['feature_index']]
    if feature_value in tree['children']:
        return predict(tree['children'][feature_value], X)
    else:
        return None 

def accuracy(tree, X, y):
    predictions = [predict(tree, x) for x in X]
    return np.mean(predictions == y)

car_data = pd.read_csv("car.data", header=None)
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
car_data.columns = columns
label_encoder = LabelEncoder()

# Apply encoding to all columns, including target 'evaluation'
for column in car_data.columns:
    car_data[column] = label_encoder.fit_transform(car_data[column])

X = car_data.drop('evaluation',axis=1).values
y = car_data['evaluation'].values

tree_1 = build_tree(X, y, max_depth=4)
tree_2 = build_tree(X, y, max_depth=5)
tree_3 = build_tree(X, y, max_depth=6)
tree_4 = build_tree(X, y, max_depth=7)

acc_1 = accuracy(tree_1, X, y)
acc_2 = accuracy(tree_2, X, y)
acc_3 = accuracy(tree_3, X, y)
acc_4 = accuracy(tree_4, X, y)

print(f"Accuracy for depth 4: {acc_1 * 100:.2f}%")
print(f"Accuracy for depth 5: {acc_2 * 100:.2f}%")
print(f"Accuracy for depth 6: {acc_3 * 100:.2f}%")
print(f"Accuracy for depth 7: {acc_4 * 100:.2f}%")

Accuracy for depth 4: 89.93%
Accuracy for depth 5: 96.41%
Accuracy for depth 6: 100.00%
Accuracy for depth 7: 100.00%


# Summary #
Comparing this to the previous notebook, it might be suprising, but there are a few things to consider:
- While we achieved 100% at depth 6, thats a huge issue as we are now overfitting, and this model will most likely fail in production
- The model in notebook 1 has a random_state parameter, used to improve effiecency.
- While this model does have a good accuracy, this is over test data, and is never the end all be all. It could be that this model perfomrs well now, but cannot generalize. 